# Ring spot helper demonstration


This notebook walks through the `spice.utils.ring_spot` helpers that model a cool umbra surrounded by a warmer plage ring.
We will generate a simple icosphere mesh, perturb its temperature and Ca abundance labels, and finally show how to
scale the Ca II IRT line depths for any surface element.


In [ ]:
import jax.numpy as jnp
import numpy as np
import matplotlib.pyplot as plt

from spice.models.mesh_generation import icosphere
from spice.utils.ring_spot import (
    RingSpotConfig,
    ring_spot_weights,
    apply_ring_spot_to_labels,
    ca_irt_scale_map,
    apply_ca_irt_scaling_local,
)


## Build a toy mesh and evaluate the ring weights


In [ ]:
cfg = RingSpotConfig()

# Build a modest icosphere mesh so plots stay snappy
_, _, areas, centers = icosphere(2000)

# Treat the facet centers as surface-normal directions
n_hat = jnp.asarray(centers)
n_hat = n_hat / jnp.linalg.norm(n_hat, axis=1, keepdims=True)

# Point the spot at the +Z axis
s_hat = jnp.array([0.0, 0.0, 1.0])

w_umb, w_plage = ring_spot_weights(n_hat, s_hat, cfg)

n_hat_np = np.asarray(n_hat)
lon = np.degrees(np.arctan2(n_hat_np[:, 1], n_hat_np[:, 0]))
lat = np.degrees(np.arcsin(n_hat_np[:, 2]))


In [ ]:
fig, axes = plt.subplots(1, 2, figsize=(12, 4), constrained_layout=True)
sc0 = axes[0].scatter(lon, lat, c=np.array(w_umb), s=10, cmap='magma', vmin=0.0)
axes[0].set_title('Umbra weight')
axes[0].set_xlabel('Longitude [deg]')
axes[0].set_ylabel('Latitude [deg]')
fig.colorbar(sc0, ax=axes[0], label='w_umb')

sc1 = axes[1].scatter(lon, lat, c=np.array(w_plage), s=10, cmap='viridis', vmin=0.0)
axes[1].set_title('Plage ring weight')
axes[1].set_xlabel('Longitude [deg]')
axes[1].set_ylabel('Latitude [deg]')
fig.colorbar(sc1, ax=axes[1], label='w_plage')

plt.show()


## Apply the spot to per-element labels


In [ ]:
labels = {
    'T_eff': jnp.full(n_hat.shape[0], 5700.0),
    'Ca_H': jnp.full(n_hat.shape[0], -0.1),
}

spotted = apply_ring_spot_to_labels(labels, n_hat, s_hat, cfg)

temp_delta = np.array(spotted['T_eff'] - labels['T_eff'])
ca_delta = np.array(spotted['Ca_H'] - labels['Ca_H'])

fig, axes = plt.subplots(1, 2, figsize=(12, 4), constrained_layout=True)
sc0 = axes[0].scatter(lon, lat, c=temp_delta, s=10, cmap='coolwarm')
axes[0].set_title('Temperature perturbation [K]')
axes[0].set_xlabel('Longitude [deg]')
axes[0].set_ylabel('Latitude [deg]')
fig.colorbar(sc0, ax=axes[0], label='ΔT')

sc1 = axes[1].scatter(lon, lat, c=ca_delta, s=10, cmap='Spectral')
axes[1].set_title('Ca abundance perturbation [dex]')
axes[1].set_xlabel('Longitude [deg]')
axes[1].set_ylabel('Latitude [deg]')
fig.colorbar(sc1, ax=axes[1], label='Δ[Ca/H]')

plt.show()


## Ca II IRT scaling factors


In [ ]:
scale_map = ca_irt_scale_map(n_hat, s_hat, cfg)

fig, axes = plt.subplots(1, 2, figsize=(12, 4), constrained_layout=True)
axes[0].hist(np.array(scale_map), bins=30, color='tab:blue')
axes[0].set_xlabel('Scale factor')
axes[0].set_ylabel('Count')
axes[0].set_title('Distribution of Ca II IRT scaling')

sc = axes[1].scatter(lon, lat, c=np.array(scale_map), s=10, cmap='plasma')
axes[1].set_title('Spatial map of scaling')
axes[1].set_xlabel('Longitude [deg]')
axes[1].set_ylabel('Latitude [deg]')
fig.colorbar(sc, ax=axes[1], label='S_CaIRT')

plt.show()


## Applying the scaling to a toy line profile


In [ ]:
wavelength = jnp.linspace(848.0, 870.0, 800)
I_cont = jnp.ones_like(wavelength)
I_lambda = I_cont - 0.45 * jnp.exp(-0.5 * ((wavelength - 854.2) / 0.45) ** 2)
mask_irt = (wavelength > 852.5) & (wavelength < 856.0)

# Grab the scale factor at the peak of the plage ring
scale_example = float(scale_map[np.argmax(np.array(w_plage))])
I_scaled = apply_ca_irt_scaling_local(I_lambda, I_cont, scale_example, mask_irt)

plt.figure(figsize=(8, 4))
plt.plot(wavelength, I_lambda, label='Original profile')
plt.plot(wavelength, I_scaled, label=f'Scaled (S={scale_example:.2f})')
plt.xlabel('Wavelength [nm]')
plt.ylabel('Normalized intensity')
plt.title('Ca II IRT depth scaling example')
plt.legend()
plt.show()
